In [16]:
import numpy 
import sklearn 
from sklearn import datasets
import syft 
import diffprivlib
import torch 
from torch import nn
from torch import optim
import pandas as pd
import time
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,auc,average_precision_score,balanced_accuracy_score,brier_score_loss,classification_report,cohen_kappa_score,confusion_matrix,dcg_score,det_curve,f1_score,fbeta_score,hamming_loss,hinge_loss,jaccard_score,log_loss,matthews_corrcoef,multilabel_confusion_matrix,ndcg_score,precision_recall_curve,precision_recall_fscore_support,precision_score,recall_score,roc_auc_score,roc_curve,top_k_accuracy_score,zero_one_loss

In [17]:
combined = pd.read_csv('Combined.csv')
combinedDWT = pd.read_csv('Combined DWT.csv')
combinedButterworth = pd.read_csv('Combined Butterworth.csv')

In [18]:
combined = combined.sample(frac=1).reset_index(drop=True)
combinedDWT = combinedDWT.sample(frac=1).reset_index(drop=True)
combinedButterworth = combinedButterworth.sample(frac=1).reset_index(drop=True)

In [19]:
data = combined[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = combined[['Diagnosis']]

def sweep(eps=[0.00001, 0.001, 0.01, 10, 1000, 10000],modelname='GNB'):
    data_train, data_test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels,random_state=100)
    accuracy = []
    precision = []
    fbeta = []
    recall = []
    roc_auc = []
    brier_loss = []
    train_time = []
    test_time = []
    epsilon = []
    for i in range(len(eps)):
        model = ''
        if modelname=='GNB':
            model = diffprivlib.models.GaussianNB(epsilon=eps[i])
        elif modelname == 'LRG':
            model = diffprivlib.models.LogisticRegression(epsilon=eps[i])
        elif modelname == 'RFC':
            model = diffprivlib.models.RandomForestClassifier(random_state=1,epsilon=eps[i])
        elif modelname == 'DTC':
            model = diffprivlib.models.DecisionTreeClassifier(random_state=1,epsilon=eps[i])
        else:
            print('Model name not correct')
            return None

        a = time.time()
        model.fit(data_train, labels_train)
        b = time.time()
        c = time.time()
        pred = model.predict(data_test)
        d = time.time()
        #pred = pd.Categorical(pred).codes
        #print('Pred:')
        #print(pred)
        #labels_test = pd.Categorical(labels_test['Diagnosis']).codes
        #print('Label:')
        #print(labels_test)
        train_time.append(b-a)
        test_time.append(d-c)
        epsilon.append(str(eps[i]))
        accuracy.append(accuracy_score(labels_test, pred))
        fbeta.append(fbeta_score(labels_test, pred, average='macro', beta=0.5))
        precision.append(precision_score(labels_test, pred, average='macro'))
        recall.append(recall_score(labels_test, pred, average='macro'))
        #roc_auc.append(roc_auc_score(labels_test, pred))
        #brier_loss.append(brier_score_loss(labels_test, pred))
    values = [epsilon,accuracy,fbeta,precision,recall,train_time,test_time]
    """values = {
        'Epsilon': epsilon,
        #'Model': 'GaussianNB',
        'Accuracy': accuracy,
        'FBeta': fbeta,
        'Precision': precision,
        'Recall': recall,
        #'ROC AOC': roc_auc,
        #'Brier Loss': brier_loss
    }"""
    return values

eps=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000] # Try changing these values to see how the accuracy plot changes!

metrics = sweep(eps,'GNB')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'GaussianNB'
gnb = df

metrics = sweep(eps,'LRG')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'LogisticRegression'
lrg = df

"""metrics = sweep(eps,'RFC')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'Random Forest'
rfc = df

metrics = sweep(eps,'DTC')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'Decision Tree'
dtc = df"""

raw = pd.concat([gnb,lrg],axis=0)
raw['Data Type'] = 'Raw'
raw

/home/subhrangshu/anaconda3/envs/Federated-Learning/lib/python3.10/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/subhrangshu/anaconda3/envs/Federated-Learning/lib/python3.10/site-packages/diffprivlib/models/naive_bayes.py:107: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/home/subhrangshu/anaconda3/envs/Federated-Learning/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples

,1e-05,0.0001,0.001,0.01,0.1,1,10,100,1000,10000,Algorithm,Data Type
Metrics,,,,,,,,,,,,
Accuracy,0.24682,0.738531,0.24682,0.680729,0.751456,0.829387,0.830524,0.830317,0.830421,0.830421,GaussianNB,Raw
FBeta,0.145297,0.393289,0.145297,0.468519,0.618691,0.768487,0.77131,0.771026,0.771171,0.771171,GaussianNB,Raw
Precision,0.12341,0.374755,0.12341,0.474825,0.642851,0.773008,0.772777,0.772491,0.772631,0.772631,GaussianNB,Raw
Recall,0.5,0.490275,0.5,0.485514,0.590298,0.753706,0.765868,0.76559,0.765752,0.765752,GaussianNB,Raw
Train Time,60.486689,0.200771,0.136941,0.111977,0.10925,0.106429,0.100889,0.10544,0.117998,0.106398,GaussianNB,Raw
Test Time,0.002459,0.003,0.003397,0.003091,0.002464,0.002837,0.002372,0.002916,0.002707,0.004086,GaussianNB,Raw
Accuracy,0.75318,0.704064,0.65381,0.666081,0.685796,0.751491,0.749388,0.74925,0.749354,0.749285,LogisticRegression,Raw
FBeta,0.396145,0.426279,0.426382,0.575154,0.512715,0.548683,0.540137,0.539632,0.53967,0.539575,LogisticRegression,Raw
Precision,0.37659,0.433052,0.425153,0.574578,0.519404,0.626969,0.616819,0.616153,0.616556,0.616267,LogisticRegression,Raw


In [20]:
data = combinedDWT[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = combinedDWT[['Diagnosis']]

def sweep(eps=[0.00001, 0.001, 0.01, 10, 1000, 10000],modelname='GNB'):
    data_train, data_test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels,random_state=100)
    accuracy = []
    precision = []
    fbeta = []
    recall = []
    roc_auc = []
    brier_loss = []
    epsilon = []
    train_time = []
    test_time = []
    for i in range(len(eps)):
        model = ''
        if modelname=='GNB':
            model = diffprivlib.models.GaussianNB(epsilon=eps[i])
        elif modelname == 'LRG':
            model = diffprivlib.models.LogisticRegression(epsilon=eps[i])
        elif modelname == 'RFC':
            model = diffprivlib.models.RandomForestClassifier(n_estimators=100, random_state=0,epsilon=eps[i])
        elif modelname == 'DTC':
            model = diffprivlib.models.DecisionTreeClassifier(random_state=1,epsilon=eps[i])
        else:
            print('Model name not correct')
            return None

        a = time.time()
        model.fit(data_train, labels_train)
        b = time.time()
        c = time.time()
        pred = model.predict(data_test)
        d = time.time()
        #pred = pd.Categorical(pred).codes
        #print('Pred:')
        #print(pred)
        #labels_test = pd.Categorical(labels_test['Diagnosis']).codes
        #print('Label:')
        #print(labels_test)
        train_time.append(b-a)
        test_time.append(d-c)
        epsilon.append(str(eps[i]))
        accuracy.append(accuracy_score(labels_test, pred))
        fbeta.append(fbeta_score(labels_test, pred, average='macro', beta=0.5))
        precision.append(precision_score(labels_test, pred, average='macro'))
        recall.append(recall_score(labels_test, pred, average='macro'))
        #roc_auc.append(roc_auc_score(labels_test, pred))
        #brier_loss.append(brier_score_loss(labels_test, pred))
    values = [epsilon,accuracy,fbeta,precision,recall,train_time,test_time]
    """values = {
        'Epsilon': epsilon,
        #'Model': 'GaussianNB',
        'Accuracy': accuracy,
        'FBeta': fbeta,
        'Precision': precision,
        'Recall': recall,
        #'ROC AOC': roc_auc,
        #'Brier Loss': brier_loss
    }"""
    return values

eps=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000] # Try changing these values to see how the accuracy plot changes!

metrics = sweep(eps,'GNB')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'GaussianNB'
gnb = df

metrics = sweep(eps,'LRG')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'LogisticRegression'
lrg = df

"""metrics = sweep(eps,'RFC')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'Random Forest'
rfc = df

metrics = sweep(eps,'DTC')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'Decision Tree'
dtc = df"""

dwt = pd.concat([gnb,lrg],axis=0)
dwt['Data Type'] = 'DWT'
dwt

/home/subhrangshu/anaconda3/envs/Federated-Learning/lib/python3.10/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/subhrangshu/anaconda3/envs/Federated-Learning/lib/python3.10/site-packages/diffprivlib/models/naive_bayes.py:107: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/home/subhrangshu/anaconda3/envs/Federated-Learning/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples

,1e-05,0.0001,0.001,0.01,0.1,1,10,100,1000,10000,Algorithm,Data Type
Metrics,,,,,,,,,,,,
Accuracy,0.247165,0.752835,0.725399,0.786613,0.829973,0.88388,0.887085,0.887602,0.887568,0.887568,GaussianNB,DWT
FBeta,0.145488,0.395993,0.39058,0.728338,0.780897,0.847419,0.852113,0.85285,0.852805,0.852805,GaussianNB,DWT
Precision,0.123583,0.376417,0.372931,0.724556,0.775871,0.855873,0.860729,0.86145,0.861415,0.861415,GaussianNB,DWT
Recall,0.5,0.5,0.481778,0.762269,0.82769,0.821062,0.825251,0.826016,0.825946,0.825946,GaussianNB,DWT
Train Time,79.148934,6.674312,0.155501,0.122466,0.101879,0.103122,0.108585,0.101325,0.101835,0.101636,GaussianNB,DWT
Test Time,0.002415,0.002356,0.002404,0.00249,0.002446,0.003006,0.002764,0.002624,0.002413,0.002393,GaussianNB,DWT
Accuracy,0.247165,0.247165,0.641126,0.839693,0.816427,0.864095,0.872781,0.872368,0.870748,0.870748,LogisticRegression,DWT
FBeta,0.145488,0.145488,0.587626,0.791812,0.743486,0.836139,0.850023,0.850024,0.847464,0.847464,LogisticRegression,DWT
Precision,0.123583,0.123583,0.592415,0.90572,0.768231,0.910435,0.915081,0.917614,0.916677,0.916677,LogisticRegression,DWT


In [21]:
data = combinedButterworth[['Ax','Ay','Az','Gx','Gy','Gz']]
labels = combinedButterworth[['Diagnosis']]

def sweep(eps=[0.00001, 0.001, 0.01, 10, 1000, 10000],modelname='GNB'):
    data_train, data_test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels,random_state=100)
    accuracy = []
    precision = []
    fbeta = []
    recall = []
    roc_auc = []
    brier_loss = []
    epsilon = []
    train_time = []
    test_time = []
    for i in range(len(eps)):
        model = ''
        if modelname=='GNB':
            model = diffprivlib.models.GaussianNB(epsilon=eps[i])
        elif modelname == 'LRG':
            model = diffprivlib.models.LogisticRegression(epsilon=eps[i])
        elif modelname == 'RFC':
            model = diffprivlib.models.RandomForestClassifier(n_estimators=100, random_state=0,epsilon=eps[i])
        elif modelname == 'DTC':
            model = diffprivlib.models.DecisionTreeClassifier(random_state=1,epsilon=eps[i])
        else:
            print('Model name not correct')
            return None
        a = time.time()
        model.fit(data_train, labels_train)
        b = time.time()
        c = time.time()
        pred = model.predict(data_test)
        d = time.time()
        #pred = pd.Categorical(pred).codes
        #print('Pred:')
        #print(pred)
        #labels_test = pd.Categorical(labels_test['Diagnosis']).codes
        #print('Label:')
        #print(labels_test)
        epsilon.append(str(eps[i]))
        accuracy.append(accuracy_score(labels_test, pred))
        fbeta.append(fbeta_score(labels_test, pred, average='macro', beta=0.5))
        precision.append(precision_score(labels_test, pred, average='macro'))
        recall.append(recall_score(labels_test, pred, average='macro'))
        train_time.append(b-a)
        test_time.append(d-c)
        #roc_auc.append(roc_auc_score(labels_test, pred))
        #brier_loss.append(brier_score_loss(labels_test, pred))
    values = [epsilon,accuracy,fbeta,precision,recall,train_time,test_time]
    """values = {
        'Epsilon': epsilon,
        #'Model': 'GaussianNB',
        'Accuracy': accuracy,
        'FBeta': fbeta,
        'Precision': precision,
        'Recall': recall,
        #'ROC AOC': roc_auc,
        #'Brier Loss': brier_loss
    }"""
    return values

eps=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000] # Try changing these values to see how the accuracy plot changes!

metrics = sweep(eps,'GNB')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'GaussianNB'
gnb = df

metrics = sweep(eps,'LRG')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'LogisticRegression'
lrg = df

"""metrics = sweep(eps,'RFC')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'Random Forest'
rfc = df

metrics = sweep(eps,'DTC')
df = pd.DataFrame(metrics)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df['Metrics'] = ['Accuracy','FBeta','Precision','Recall','Train Time','Test Time']
df = df.set_index('Metrics')
df['Algorithm'] = 'Decision Tree'
dtc = df"""

bw = pd.concat([gnb,lrg],axis=0)
bw['Data Type'] = 'BW'
bw

/home/subhrangshu/anaconda3/envs/Federated-Learning/lib/python3.10/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/subhrangshu/anaconda3/envs/Federated-Learning/lib/python3.10/site-packages/diffprivlib/models/naive_bayes.py:107: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/home/subhrangshu/anaconda3/envs/Federated-Learning/lib/python3.10/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

,1e-05,0.0001,0.001,0.01,0.1,1,10,100,1000,10000,Algorithm,Data Type
Metrics,,,,,,,,,,,,
Accuracy,0.739737,0.459725,0.338124,0.767932,0.811533,0.822597,0.821804,0.821184,0.821149,0.821149,GaussianNB,BW
FBeta,0.436029,0.493494,0.373202,0.66752,0.743675,0.758238,0.757287,0.756248,0.7562,0.7562,GaussianNB,BW
Precision,0.499436,0.567409,0.467763,0.680959,0.748772,0.767386,0.765679,0.764972,0.764913,0.764913,GaussianNB,BW
Recall,0.499931,0.576663,0.470136,0.64067,0.727792,0.73274,0.733419,0.731707,0.731684,0.731684,GaussianNB,BW
Train Time,0.880714,1.666841,0.249369,0.102694,0.098618,0.10829,0.130749,0.09811,0.126721,0.120502,GaussianNB,BW
Test Time,0.00246,0.002596,0.005092,0.002375,0.002362,0.003235,0.002405,0.002377,0.002909,0.002815,GaussianNB,BW
Accuracy,0.248509,0.751491,0.710992,0.754834,0.809499,0.83859,0.840416,0.840554,0.840658,0.840623,LogisticRegression,BW
FBeta,0.146233,0.395397,0.51077,0.670932,0.721581,0.784963,0.789205,0.789522,0.789724,0.78966,LogisticRegression,BW
Precision,0.124255,0.375745,0.531376,0.671204,0.82868,0.842859,0.853304,0.854004,0.854202,0.854167,LogisticRegression,BW


In [22]:
final = pd.concat([raw,dwt,bw],axis=0)
final

,1e-05,0.0001,0.001,0.01,0.1,1,10,100,1000,10000,Algorithm,Data Type
Metrics,,,,,,,,,,,,
Accuracy,0.24682,0.738531,0.24682,0.680729,0.751456,0.829387,0.830524,0.830317,0.830421,0.830421,GaussianNB,Raw
FBeta,0.145297,0.393289,0.145297,0.468519,0.618691,0.768487,0.77131,0.771026,0.771171,0.771171,GaussianNB,Raw
Precision,0.12341,0.374755,0.12341,0.474825,0.642851,0.773008,0.772777,0.772491,0.772631,0.772631,GaussianNB,Raw
Recall,0.5,0.490275,0.5,0.485514,0.590298,0.753706,0.765868,0.76559,0.765752,0.765752,GaussianNB,Raw
Train Time,60.486689,0.200771,0.136941,0.111977,0.10925,0.106429,0.100889,0.10544,0.117998,0.106398,GaussianNB,Raw
Test Time,0.002459,0.003,0.003397,0.003091,0.002464,0.002837,0.002372,0.002916,0.002707,0.004086,GaussianNB,Raw
Accuracy,0.75318,0.704064,0.65381,0.666081,0.685796,0.751491,0.749388,0.74925,0.749354,0.749285,LogisticRegression,Raw
FBeta,0.396145,0.426279,0.426382,0.575154,0.512715,0.548683,0.540137,0.539632,0.53967,0.539575,LogisticRegression,Raw
Precision,0.37659,0.433052,0.425153,0.574578,0.519404,0.626969,0.616819,0.616153,0.616556,0.616267,LogisticRegression,Raw


In [23]:
final.to_csv('Differential performance.csv')